## クラウド設定

一度だけ実行すれば良い.

In [ ]:
from pathlib import Path

config = """[default]
url=https://api.qiqb-cloud.jp
api_token=YOUR_API_TOKEN_HERE
"""
Path("~/.oqtopus").expanduser().write_text(config)
print("Configuration saved.")

Configuration saved.


QIQBクラウドのリンク：https://www.qiqb-cloud.jp/  
  
「ジョブ」のタブで, 実行済み・実行中のジョブを確認することができる. 

## ジョブの実行

以下の`PROGRAM = `に実行したい.pyファイル名を記入する.  
実行がなかなか終わらない場合はカーネル再起動が必要かもしれない.  
`payloads`に測定結果が返ってくる.  

In [2]:
from quri_parts_oqtopus.backend import OqtopusSseBackend
from oqtopus_sse_pulse.adapter import QuriAdapter, collect_payloads_from_job, save_calib_note_from_job

ラビ振動を実験し、ラビパラメータを取得する。その後に取得したデータを利用してHPIパルスを作成する.


In [ ]:
# ジョブの実行

DEVICE = "anemone"
PROGRAM = "get_calib_note"

backend = OqtopusSseBackend()

job = backend.run_sse(
    file_path=f"program/{PROGRAM}.py",
    device_id=f"{DEVICE}-pulse",
    name=PROGRAM,
)
print(f"job_id = {job.job_id}")

job.wait_for_completion()
print(f"job_status = {job.status}  elapsed={job.execution_time}")

adapter = QuriAdapter(backend)

payloads = collect_payloads_from_job(
    backend=adapter,
    job_id=job.job_id,
)

print("sample:", payloads[:3], f"... len={len(payloads)}")


job_id = 06913fe3-c61a-7a5e-8000-63e5e8a80b1c
job_status = succeeded  elapsed=30.998
sample: [{'mode': '', 'data': {'Q36': {'raw': {'I': [-2.1138060092926025, -2.353761672973633, -2.5366406440734863, -2.7077291011810303, -2.8510334491729736, -3.0286409854888916, -3.0740115642547607, -3.087764024734497, -2.959832191467285, -2.7912282943725586, -2.637314796447754, -2.4224228858947754, -2.2444510459899902, -2.0805282592773438, -2.027541160583496, -2.0371804237365723, -2.096630096435547, -2.1674656867980957, -2.3496084213256836, -2.466897487640381, -2.6876792907714844, -2.7987470626831055, -2.97562837600708, -2.9949686527252197, -3.1323468685150146, -3.0173909664154053, -2.8730263710021973, -2.804241418838501, -2.5943799018859863, -2.38612961769104, -2.2439303398132324, -2.1174564361572266, -2.0479564666748047, -2.0073671340942383, -2.071668863296509, -2.1285548210144043, -2.2767281532287598, -2.404033660888672, -2.670067548751831, -2.7765636444091797, -3.0013489723205566, -2.9468545913696

# Calibration Noteの取得
以下のコードでextracedディレクトリにcalib_note.jsonが保存される.calib_noteの取得方法はget_calib_note.pyを参照のこと.
```python
save_calib_note_from_job(
	backend=adapter,
	job_id=job.job_id,
)
```

取得したjsonファイルを以下の自分のリポジトリにアップロードしておくと,そのパラメータを利用してHPIパルスを作成できる.
ファイルをGitHubにアップロード後最大2分程度で反映される.
https://github.com/orangekame3/oqtopus-pulse-config/


実験時には以下のようにparams_dirとcalib_note_pathを指定してExperimentを作成する.
```python
exp = Experiment(
	chip_id="64Qv3",
	muxes=[9],
	params_dir="/sse/in/repo/miyanaga/params",
	calib_note_path="/sse/in/repo/miyanaga/calib_note.json"
)
```

In [4]:
save_calib_note_from_job(
	backend=adapter,
	job_id=job.job_id,
)

PosixPath('extracted/calib_note.json')

HPIパルスを作成する。

In [ ]:
# ジョブの実行

DEVICE = "anemone"
PROGRAM = "create_hpi"

backend = OqtopusSseBackend()

job = backend.run_sse(
    file_path=f"program/{PROGRAM}.py",
    device_id=f"{DEVICE}-pulse",
    name=PROGRAM,
)
print(f"job_id = {job.job_id}")

job.wait_for_completion()
print(f"job_status = {job.status}  elapsed={job.execution_time}")

adapter = QuriAdapter(backend)

payloads = collect_payloads_from_job(
    backend=adapter,
    job_id=job.job_id,
)
save_calib_note_from_job(
    backend=adapter,
    job_id=job.job_id,
)
print("sample:", payloads[:3], f"... len={len(payloads)}")


job_id = 06913fe6-89c7-7ee4-8000-bcd363680f57
job_status = succeeded  elapsed=17.53
sample: [{'mode': '', 'data': {'Q36': {'raw': {'I': [-3.004467248916626, -2.983452796936035, -2.9042539596557617, -2.82334041595459, -2.627967119216919, -2.432013988494873, -2.303433418273926, -2.1011061668395996, -1.9960893392562866, -2.0158095359802246, -2.047173500061035, -2.1822590827941895, -2.295870780944824, -2.4026026725769043, -2.5306174755096436, -2.639944076538086, -2.752089500427246, -2.789905548095703, -2.747622489929199, -2.7676234245300293], 'Q': [-4.021335601806641, -3.803311586380005, -3.9853057861328125, -4.151266098022461, -4.495615482330322, -4.894684791564941, -5.2135396003723145, -5.705432891845703, -5.867847442626953, -5.954934120178223, -5.827388286590576, -5.4577131271362305, -5.254084587097168, -4.9606709480285645, -4.754789352416992, -4.479434013366699, -4.326351165771484, -4.119341850280762, -4.435734748840332, -4.531561851501465]}, 'normalized': {'I': [3.0413150787353516, 2.